## Import Packages

In [1]:
import kagglehub
import numpy as np
import pandas as pd
import os

## Load the dataset

In [2]:
metadata_df = pd.read_csv('data/andrewmvd/isic-2019/versions/1/ISIC_2019_Training_Metadata.csv')


print("ISIC_2019_Training_Metadata.csv")
print(metadata_df.head(),"\n")
print("Number of rows in MetaData CSV:", len(metadata_df))

ISIC_2019_Training_Metadata.csv
          image  age_approx anatom_site_general lesion_id     sex
0  ISIC_0000000        55.0      anterior torso       NaN  female
1  ISIC_0000001        30.0      anterior torso       NaN  female
2  ISIC_0000002        60.0     upper extremity       NaN  female
3  ISIC_0000003        30.0     upper extremity       NaN    male
4  ISIC_0000004        80.0     posterior torso       NaN    male 

Number of rows in MetaData CSV: 25331
